---
# Day 10: Pipe Maze
---

In [1]:
from typing import List, Dict, Tuple, Set
import numpy as np

In [2]:
V = lambda *x : np.array(x)

## Load data

In [3]:
def read_data(file_tag: str) -> List[str]:
    with open(f"data/day10_{file_tag}.txt", "r") as f:
        data = f.read().splitlines()
    return data

In [4]:
file_tag = "input" #"input_test4"
data = read_data(file_tag)

## --- Part One ---

In [5]:
PIPE_DIR_MAP = {
    "|": [V(1,0), V(-1,0)], 
    "-": [V(0,1), V(0,-1)], 
    "J": [V(-1,0), V(0,-1)], 
    "L": [V(-1,0), V(0, 1)],
    "7": [V(1,0), V(0,-1)], 
    "F": [V(1,0), V(0,1)]
}

        
def check_path(maze: Dict[Tuple[int, int], List[Tuple[int, int]]], path: List[Tuple[int, int]]) -> List[Tuple[int, int]]:
    if not path[-1] in maze:
        return []
    while True:
        choices = maze[path[-1]]
        targets = set(choices).difference(set(path))
        if len(targets) != 1:
            if len(targets) == 0 and path[0] in choices: # start is the only choice
                return path
            else:
                return []
        path.append(list(targets)[0])        

    
def find_maze_ways(data: List[str]) -> (Dict[Tuple[int, int], List[Tuple[int, int]]], Tuple[int, int]):
    pipes = {}
    starting_position = None
    for i, row in enumerate(data):
        for j, col in enumerate(list(row)):
            if col == ".":
                pass
            elif col == "S":
                starting_position = (i, j)
            else:
                destinations = [tuple((V(i,j) + PIPE_DIR_MAP[col][ii]).tolist()) for ii in range(2)]
                pipes[(i,j)] = destinations
  
    return pipes, starting_position

In [6]:
pipes, starting_position = find_maze_ways(data)

In [7]:
for dir in [(-1, 0), (0, 1), (1, 0), (0, -1)]:
    next_step = tuple((V(*starting_position) + V(*dir)).tolist())

    path_start = [starting_position, next_step]
    path = check_path(pipes, path_start)
    if len(path) > 0:
        print("Path found!")
        v1 = V(*dir)
        v2 = V(*path[-1]) - V(*path[0])
        break        

Path found!


In [8]:
farthest_distance = int(len(path)/2)
print(f"The farthest distance from the start is {farthest_distance}.")

The farthest distance from the start is 6786.


In [9]:
start_pipe = [
    s for s, v in PIPE_DIR_MAP.items() 
    if (np.array_equal(v[0], v1) and np.array_equal(v[1], v2)) or (np.array_equal(v[0], v2) and np.array_equal(v[1], v1))
][0]

data = [row.replace("S",start_pipe) for row in data]

print(f"The pipe at the start point is {start_pipe}.")

The pipe at the start point is |.


## --- Part Two ---

In [10]:
grid = ["".join([ch if (i,j) in path else "." for j, ch in enumerate(row)]) for i, row in enumerate(data)]

In [11]:
n_enclosed = 0
loop_set = set(path)
for i, row in enumerate(grid[1:-1]):
    n_crossed_lines = 0
    for j, col in enumerate(row):
        on_loop = (i,j) in loop_set
        if (not on_loop) and n_crossed_lines % 2 == 1:
            n_enclosed += 1
        elif on_loop and col in "|LJ":
            n_crossed_lines += 1
    prev_row = row

In [12]:
print(n_enclosed)

495


#### Alternative solution

In [13]:
def find_empty_tiles(data: List[str], path: List[Tuple[int, int]]) -> Set[Tuple[int, int]]:
    tiles = []
    loop_set = set(path)
    for i, row in enumerate(data):
        for j, col in enumerate(row):
            if not(i, j) in loop_set:
                tiles.append((i,j))
    return set(tiles)

In [14]:
def collect_right_elements(path: List[Tuple[int, int]]) -> (Set[Tuple[int, int]], Set[Tuple[int, int]]):
    right_elements = set()
    for k, p in enumerate(path):
        i, j = p
        dir = V(*path[(k+1)%len(path)]) - V(*path[k])
        if np.array_equal(dir, V(1, 0)): #going down
            right_elements.add((i,j-1))
        elif np.array_equal(dir, V(-1, 0)): #going up
            right_elements.add((i,j+1))
        elif np.array_equal(dir, V(0, 1)): #going right    
            right_elements.add((i+1,j))
        elif np.array_equal(dir, V(0, -1)): #going left    
            right_elements.add((i-1,j))
    return right_elements

In [15]:
def expand_tile_set(in_set: Set[Tuple[int, int]], full_set: Set[Tuple[int, int]]) -> Set[Tuple[int, int]]:
    m = len(data)
    n = len(data[0])
    new_tile_count = -1
    while (new_tile_count != 0):
        new_tiles = set()
        for i, j in in_set:
            # down
            if i < m -1 and (i+1, j) in full_set:
                new_tiles.add((i+1, j))
            # up
            if i >= 1 and (i-1, j) in full_set:
                new_tiles.add((i-1, j))
            # right
            if j < n -1 and (i, j+1) in full_set:
                new_tiles.add((i, j+1))
            # left
            if j >= 1 and (i, j-1) in full_set:
                new_tiles.add((i, j-1))
        new_tiles = new_tiles.difference(in_set)
        new_tile_count = len(new_tiles)
        in_set = in_set.union(new_tiles)
    return in_set       

In [16]:
empty_tiles = find_empty_tiles(data, path)

In [17]:
res = collect_right_elements(path)
res = expand_tile_set(res.intersection(empty_tiles), empty_tiles)
res_c = empty_tiles.difference(res)

print(f"Size of set of the elements to the right of the loop: {len(res)}.")
print(f"-> complementary: {len(res_c)}.")

Size of set of the elements to the right of the loop: 491.
-> complementary: 5537.
